# 変量heightのクレンジング

ここからは各変量について個別にクレンジングを実施します．

先ず，前提として次のプログラムを実施します．

- ライブラリーの搬入
- CSVファイルを読み込んでデータフレームを生成
- 全角半角変換ツールの準備
- 小数点付数を判別する正規表現のコンパイル


In [1]:
import re
import unicodedata
import pandas as pd

In [2]:
df = pd.read_csv('original_body_data.csv')
df.set_index('person',inplace=True)

In [3]:
floatCheck = re.compile(r'^\s*([+-]?(\d+\.?\d*|\.\d+))\s*$')

*****
## データフレームの基礎情報

まずはデータフレームの行数を確認します．
そして，データフレームの一部を表示します．

```Python
len(df)
df.head(5)
```

In [4]:
len(df)

200

In [5]:
df.head(5)

,height,weight,age,gender,blood
person,,,,,
p001,157.7,56.8,20,Male,B
p002,169.7,58.1,53,Female,A
p003,160.6,96.8,22,Male,O
p004,'155.8',,,Female,O
p005,115.5,22.5,7,Male,NaN


*****
## 変量heightの概要

変量の概要は<font color=green>describe()</font>メソッドを発行することによって得ることができます．

```Python
df.height.describe()
```

In [6]:
df.height.describe()

count       200
unique      160
top       160.7
freq          4
Name: height, dtype: object

describe()メソッドによって基礎統計が得られます．
この結果は後ほど使うことになりますが，ここでは最後の行に注目します．

> 
```
Name: height, dtype: object
```

これは，df.heightの各データのデータ型がobject型であることを示しています．
object型はpandasにある特別なデータ型でオブジェクトへのポインターを格納しています．
各データはそれぞれ独自のデータ型を持っています．
各要素のデータ型を調べるために，リスト内包表記とtype()関数を組み合わせます．そして，その結果をset()関数で重複を除いたデータ型の取得します．

```Python
set([type(x) for x in df.height])
```

In [7]:
set([type(x) for x in df.height])

{str}

df.heightの各要素はstr型すなわち文字列であることが分かりました．

## 数値化の可能性の確認

df.heightの各要素である文字列が小数点付数として数値化可能であるかを確認する必要があります．

もし確認なしで数値への変換を<font color=green>float()</font>関数で実施したらどうなるかテストします．
まずは，df.heightの最初のデータだけを数値化してみます

```Python
float(df.height[0])
```

In [8]:
float(df.height[0])

157.7

このように数値化が成功しました．
そこで，リスト内包表記を使って，df.heightのデータ全ての数値化を試みます．
エラーが起こることを想定して実施するので，この処理を<font color=green>try:</font>ブロックで実施してエラーが発生した場合は例外処理でエラー内容をプリントするようにします．

```Python
try:
    [float(x) for x in df.height]
except ValueError as err:
    print(err)
```

In [9]:
try:
    [float(x) for x in df.height]
except ValueError as err:
    print(err)

could not convert string to float: "'155.8'"


すると，途中で数値化できない文字列がありエラーが発生しました．
ここで発生したエラーは，数値化できない文字列"'155.8'"をfloat()関数に渡したために起こったものです．
プログラムの処理はここで中断して例外処理になり，エラーメッセージを出力しました．

このように，クレンジングせずに数値化を実施することは無謀な試みとなります．
他にも数値化できないデータがあるかもしれませんので，<font color=green>floatCheck.fullmatch()</font>関数によって，数値化できないデータをリストアップします．

```Python
[x for x in df.height if not floatCheck.match(x)]
```

In [10]:
[x for x in df.height if not floatCheck.fullmatch(x)]

["'155.8'", "' 151.2 '", "'１７２．９'"]

この結果で，クレンジング対象となるデータが判明しました．
これらのデータを綺麗にするように処理を進めていきます．
この数値化できないデータを観察すると，次の事が課題であると思われます．

- 文字列がシングルクォーテーション「'」で囲われている
- 数字の前後に空白文字がある
- 全角で小数点付数が記載されている

これらを解消するようにクレンジングを実施していきます．

## シングルクォーテーションの削除

文字列がシングルクォーテーションで囲われているので，これを<font color=green>strip()</font>メソッドで削除します．
例として，文字列「'155.8'」からシングルクォーテーションを除去します．

```Python
"'155.8'".strip("'")
```

In [11]:
"'155.8'".strip("'")

'155.8'

この操作をdf.heightの全てのデータについて適用します．
その結果として，数値化できないデータを表示します．

```Python
[x for x in df.height if not floatCheck.fullmatch(x.strip("'"))]
```

In [12]:
[x for x in df.height if not floatCheck.fullmatch(x.strip("'"))]

["'１７２．９'"]

シングルクォーテーションを除去した後でも数値化できないデータは「'１７２．９'」だけとなりました．

## 全角から半角への変換

データ「'１７２．９'」について，シングルクォーテーションを除去した後に全角文字を半角に変換します．

```Python
unicodedata.normalize('NFKC',"'１７２．９'".strip("'"))
```

In [13]:
unicodedata.normalize('NFKC',"'１７２．９'".strip("'"))

'172.9'

身長データには負の数はありませんが，負の数がある場合は，この後ろにtranslate(doubleToSingle)関数を付けます．

```Python
unicodedata.normalize('NFKC',"'１７２．９'".strip("'")).translate(doubleToSingle)
```

この操作をdf.heightの全データに適用し，その結果として数値化できないデータを表示します．

```Python
[x for x in df.height if not floatCheck.fullmatch(unicodedata.normalize('NFKC',x.strip("'")))]
```

In [14]:
[x for x in df.height if not floatCheck.fullmatch(unicodedata.normalize('NFKC',x.strip("'")))]

[]

これで，df.heightの値が全て数値化可能になります．
df.height[3]を例にとって，この値に対してクレンジングして数値化するプロセスを単純なコードで書くと次のようになります．

```Python
cleansedValue = unicodedata.normalize('NFKC',df.height[3].strip("'"))
if floatCheck.fullmatch(cleansedValue):
    floatValue = float(cleansedValue)
else:
    floatValue = None
print(floatValue)
```

In [15]:
cleansedValue = unicodedata.normalize('NFKC',df.height[3].strip("'"))
if floatCheck.fullmatch(cleansedValue):
    floatValue = float(cleansedValue)
else:
    floatValue = None
print(floatValue)

155.8


これは処理を分かりやすく書いたものです．
実際は，もっとコンパクトなプログラムになります．

## データフレームでの処理

ここまでdf.heightを取り出してリスト配列としてクレンジング処理を行ってきましたが，実際にはデータフレームの更新作業となります．
よって，データフレーム内でのクレンジングを行うプログラムを作成します．
この処理は次のステップで行います．

- シングルクォーテーションを除去し全角文字を半角に変換したデータを新たにcleansedHeightという列に格納する．
- cleansedHeightの値を数値化したデータを新たにheight2という列に格納する．
- 列cleansedHeightを削除する．

この処理では文字列を綺麗にした中間データを作成することによって，プログラムが複雑になることを避けています．

### 文字列の整形

まず，df.heightの値を文字列として綺麗な形に整えます．
その結果は新しい列cleansedHeightとしてデータフレームに追加します．

```Python
df['cleansedHeight'] = [unicodedata.normalize('NFKC',x.strip("'")) for x in df.height]
```

In [16]:
df['cleansedHeight'] = [unicodedata.normalize('NFKC',x.strip("'")) for x in df.height]

### 文字列の数値化

次にdf.cleansedHeightの値を数値化します．
ただし，数値化できない値については<font color=green>None</font>で置き換えます．

```Python
df['height2'] = [float(x) if floatCheck.fullmatch(x) else None for x in df.cleansedHeight]
```

In [17]:
df['height2'] = [float(x) if floatCheck.fullmatch(x) else None for x in df.cleansedHeight]

### 不要な中間データの削除

使用済みとなったdf.cleansedHeight列を削除します．
列の削除には，drop()メソッドとdel命令の2通りの方法があります．
ここでは<font color=green>drop()</font>メソッドによる方法で実施します．

```Python
df.drop('cleansedHeight',axis=1,inplace=True)
```
あるいは
```Python
del df['cleansedHeight']
```

In [18]:
df.drop('cleansedHeight',axis=1,inplace=True)

### クレンジング済み変量height2の確認

これで身長データについてのクレンジングが終了したので，その結果を<font color=green>describe()</font>メソッドで確認します．

```Python
df.height2.describe()
```

In [19]:
df.height2.describe()

count     200.000000
mean      169.107500
std       103.481227
min        77.300000
25%       155.250000
50%       162.350000
75%       170.750000
max      1609.000000
Name: height2, dtype: float64

この出力により，200個の全てのデータが小数点付数になっていることが分かります．
念のためにデータフレームの先頭の数行を表示します．
新たにheight2という列が追加されていることが確認できます．

In [20]:
df.head(4)

,height,weight,age,gender,blood,height2
person,,,,,,
p001,157.7,56.8,20,Male,B,157.7
p002,169.7,58.1,53,Female,A,169.7
p003,160.6,96.8,22,Male,O,160.6
p004,'155.8',,,Female,O,155.8


ここでは，df.heightに対して別の列としてdf.height2を作成しました．
これは元のデータとクレンジング済みのデータを比較するためです．
実際には，オリジナルデータが別に保管されている前提で，df.heightを上書きする方が一般的です．

これで，身長のデータについてのクレンジングは完了です．

*****